## Results and Plotting
A notebook to compile results and organise plots and other resources needed for reporting

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# set global plotting params here for consistency
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

In [172]:
stim_freqs = [7,10,12] # stim freqs used
fs = 64 # sampling freq
Ns = 256 # number of sample points to consider
Nh = 1 # number of harmonics for CCA-based algos

index_pos = dict(zip(["Nc", "Ns", "Nt"], range(3)))

## Data Loading
Load data from json log files and arrange by frequency. The compiled data is stored in the dictionary `data` whose keys are the stimulus frequencies used and whose values are the data tensors corresponding to trials at those frequencies. Data tensors will be arranged like `Nc x Ns x Nt` (channels x samples x trials). 

Note that in this project, we only effectively had one channel. Also, all Nt trials would be independent recordings at the same stimulus frequency.


In [173]:
from eeg_lib.utils import read_json
import json

tests = {7: ["test-7hz-pos2"], 
         10: ["test-10hz-pos2"], 
         12: ["test-12hz-pos2"]
        }

all_data = read_json('eeg_lib/log_data.json')
data = {}

for f, test_set in tests.items():
    data[f] = []
    
    for test in test_set:
        values = all_data[test]
        proc_data = np.array([json.loads(values[i]) for i in range(len(values))])
        data[f].append(proc_data[1:, :Ns].reshape((1, Ns, -1))) # exclude first trial
        
# del all_data    

for f, proc_data in data.items():
    if len(proc_data) <= 1:
        data[f] = proc_data[0]
    else:
        data[f] = np.concatenate([*proc_data], axis=-1) # merge data from across trials

## Decoding
Run various decoding algos on gathered data and store results for comparison

### CCA
Vanilla CCA with no historical training data used across evaluations

In [174]:
from eeg_lib.cca import CCA

cca = CCA(stim_freqs, fs, Nh=Nh)

cca_results = {f:[] for f in stim_freqs}
cca_agg_results = {f:{} for f in stim_freqs}

for f in stim_freqs:
    data_f = data[f]
    for trial in range(1, data[f].shape[index_pos["Nt"]]):
        Xi = data_f[:, :, trial]
        result = cca.compute_corr(Xi)
        result = {k:np.round(v[0], 6) for k,v in result.items()}
        
        result['trial'] = f'f{f}_{trial}'
        result['y'] = f
        cca_results[f].append(result)
        
        # compute CCA result using data aggregated across trials
        agg_result = cca.compute_corr(data_f.mean(axis=index_pos["Nt"]))
        agg_result = {k:np.round(v[0], 6) for k,v in agg_result.items()}
        agg_result['y'] = f
        cca_agg_results[f] = agg_result
    
cca_df = pd.concat([pd.DataFrame(result_set) for result_set in cca_results.values()]).set_index('trial')
cca_df['y_hat'] = cca_df[stim_freqs].apply(lambda row: stim_freqs[np.argmax(row)], axis=1)

cca_agg_df = pd.DataFrame(list(cca_agg_results.values()))
cca_agg_df['y_hat'] = cca_agg_df[stim_freqs].apply(lambda row: stim_freqs[np.argmax(row)], axis=1)

cca_df

,7,10,12,y,y_hat
trial,,,,,
f7_1,0.147529,0.071546,0.018522,7,7
f7_2,0.119633,0.118277,0.074220,7,7
f7_3,0.191390,0.108617,0.030403,7,7
f7_4,0.171472,0.092428,0.082114,7,7
f7_5,0.087934,0.091173,0.105607,7,12
f10_1,0.202168,0.020386,0.045348,10,7
f10_2,0.246756,0.051406,0.112536,10,7
f10_3,0.189889,0.072409,0.182558,10,7
f10_4,0.206812,0.064271,0.081093,10,7


## Template-based Algorithms 
This section explores decoding algos that, along with potentially the artificially-generated harmonic reference, include template data based on historical 'training' data. These include GCCA, MsetCCA, TRCA and others.

In [175]:
min_trial_len = np.min([test_set.shape[-1] for test_set in data.values()])

# Nf x Nc x Ns x Nt
data_tensor = np.array([test_set[:,:,:min_trial_len] for test_set in data.values()])

print("Data tensor shape: ", data_tensor.shape)

Data tensor shape:  (3, 1, 256, 6)


In [229]:
from sklearn.model_selection import LeavePOut

N_train = 3
lpo = LeavePOut(p=N_train)

n_trials = data_tensor.shape[-1]
template_idxs = list(lpo.split(range(n_trials)))

#### GCCA
Generalised CCA aims to simultaneously maximise correlation between three sets of data: historical observations, measured signals in a new sample and the pre-constructed sinusoidal reference. As interpreted by the authors (Wong et al), the optimal spatial filters obtained through GCCA perform SSVEP signal denoising.

#### MsetCCA
MsetCCA is one extension of standard CCA that takes into account historical data instead of performing inference purely on new observations. Zhang et al propose that this is one of the reasons that standard CCA performs poorly on short time windows; it effectively over fits to localised dynamics. Furthermore, the authors suggest that exclusively using the pre-constructed sinusoidal reference set is not optimal since this artificial reference does not exclude other features from real EEG data. To circumvent this, MsetCCA seeks to optimise the reference signals used in the CCA algorithm by learning multiple linear transforms to maximise overall correlation between canonical variables over many sets of EEG data at each candidate frequency fk ∈ F. This optimisation effectively finds optimal joint spatial filters w1, . . . , wNt (over Nt trials) using only historical observations (‘training’ data). The authors claim that MsetCCA outperforms similar techniques, especially in cases with few channels and short time windows.

In [ ]:
def softmax(X):
    exps = np.exp(X)
    return exps / np.sum(exps)

def cross_entropy(X, y):
    """
    source: https://deepnotes.io/softmax-crossentropy
    
    X is the output from fully connected layer (num_examples x num_classes)
    y is labels (num_examples x 1)
    """
    m = y.shape[0]
    p = softmax(X)
    log_likelihood = -np.log(p[range(m),y])
    loss = np.sum(log_likelihood) / m
    return loss

In [393]:
from eeg_lib.cca import GCCA_SSVEP
from eeg_lib.cca import MsetCCA_SSVEP

def compute_gcca_msetcca_results(gcca, mset_cca, data_tensor, stim_freqs, template_idxs, ce_loss=True):
    
    gcca_results = {f:[] for f in stim_freqs}
    mset_cca_results = {f:[] for f in stim_freqs}

    for f_idx, f in enumerate(stim_freqs):
        for split_idx, (test_idxs, train_idxs) in enumerate(template_idxs):
            chi_train = data_tensor[:, :, :, train_idxs]

            # train models on current train-test split
            gcca.fit(chi_train)
            mset_cca.fit(chi_train)

            # extract test matrices from all test indices and compute result
            for test_idx in test_idxs:
                if test_idx in train_idxs:
                    raise ValueError("Found intersection between train and test indices")
                    
                # note: we must match the number of samples Ns in chi_train
                X_test = data_tensor[f_idx, :, :, test_idx]
                
                _idx = f'f{f}_split{split_idx+1}_test{test_idx+1}'
                test_meta = {'idx': _idx, 'y': f, 'test': test_idx, 'split': split_idx}
                
                # GCCA
                result = {k: abs(np.round(v,4)) for k,v in gcca.classify(X_test).items()}
                gcca_results[f].append({**result, **test_meta})

                # MsetCCA
                result = {k: abs(np.round(v,4)) for k,v in mset_cca.classify(X_test).items()}
                mset_cca_results[f].append({**result, **test_meta})
                
    def _prep_results_df(results):
        df = pd.concat([pd.DataFrame(result_set) for result_set in results.values()])
        df['y_hat'] = df[stim_freqs].apply(lambda row: stim_freqs[np.argmax(row)], axis=1)
        df = df.set_index(['y', 'split'])
        
        if ce_loss:
            # compute cross entropy loss
            for f_idx, f in enumerate(stim_freqs):
                result = df.loc[f, stim_freqs].apply(lambda row: cross_entropy(row.values.reshape(1, -1), np.array([f_idx])), axis=1)
                df.loc[(f, ), 'ce_loss'] = result.values
                
        df['correct'] = df.index.get_level_values(level=0) == df.y_hat

        return df

    gcca_df = _prep_results_df(gcca_results)
    mset_df = _prep_results_df(mset_cca_results)
    
    return gcca_df, mset_df

def decoding_acc(result_df):
    acc = result_df['correct'].groupby(['y', 'split']).apply(lambda x: np.sum(x)/len(x))
    acc_grouped = acc.groupby('y')
    acc_av = acc_grouped.mean().to_dict()
    acc_std = acc_grouped.std().to_dict()
    return {'raw': acc, 'mean': acc_av, 'std': acc_std}

gcca = GCCA_SSVEP(stim_freqs, fs, Nh=Nh)
mset_cca = MsetCCA_SSVEP(stim_freqs)

gcca_df, mset_cca_df = compute_gcca_msetcca_results(gcca, mset_cca, data_tensor, stim_freqs, template_idxs)
gcca_acc = decoding_acc(gcca_df)
mset_acc = decoding_acc(mset_cca_df)

print("GCCA: ", gcca_acc['mean'])
print("MsetCCA: ", mset_acc['mean'])

GCCA:  {7: 1.0, 10: 0.9833333333333332, 12: 0.85}
MsetCCA:  {7: 0.9833333333333332, 10: 0.9833333333333332, 12: 0.9166666666666667}


In [392]:
gcca_df

7      10      12                idx  test  y_hat   ce_loss  \
y  split                                                                     
7  0      0.1106  0.0153  0.0082    f7_split1_test4     3      7  1.033814   
   0      0.0725  0.0106  0.0035    f7_split1_test5     4      7  1.055462   
   0      0.0540  0.0089  0.0006    f7_split1_test6     5      7  1.066056   
   1      0.2370  0.0005  0.0124    f7_split2_test3     2      7  0.950971   
   1      0.1757  0.0048  0.0013    f7_split2_test5     4      7  0.986886   
...          ...     ...     ...                ...   ...    ...       ...   
12 18     0.0013  0.0003  0.1306  f12_split19_test2     1     12  1.013977   
   18     0.0030  0.0053  0.1986  f12_split19_test4     3     12  0.973267   
   19     0.0256  0.0004  0.0200  f12_split20_test1     0      7  1.094004   
   19     0.0010  0.0004  0.0800  f12_split20_test2     1     12  1.046450   
   19     0.0168  0.0118  0.0943  f12_split20_test3     2     12  1.045998   

          correct  
y  split           
7  0         True  
   0         True  
   0         True  
   1         True  
   1         True  
...           ...  
12 18        True  
   18        True  
   19       False  
   19        True  
   19        True  

[180 rows x 8 columns]

#### Tests to explore
Some ideas for interesting tests/factors to investigate. 

Test the effect of the following on decoding accuracy:
1. number of training trials
2. number of samples in each window (Ns)
3. number of stimulus frequencies

other miscellaneous tests:
- generalisation performance on different set of data: both with pretraining from diff sets and without
- average accuracy per stimulus frequency
- average accuracy per stimulus square configuration (wide, narrow etc) * optional
- some meausre of inter-trial consistency 
- some measure of similarity between estimated outputs: e.g. log loss that penalises similar outputs

#### 1. Acc vs number of training trials

In [401]:
n_trials = data_tensor.shape[-1]
gcca_ntr_acc = []
mset_ntr_acc = []

for n_train in range(1, n_trials):
    lpo = LeavePOut(p=n_train)
    template_idxs = list(lpo.split(range(n_trials)))

    data_tensor_tmp = data_tensor[:, :, :Ns, :]
    gcca_df, mset_cca_df = compute_gcca_msetcca_results(gcca, mset_cca, data_tensor_tmp, stim_freqs, template_idxs)

    _gcca_acc = decoding_acc(gcca_df)
    _mset_acc = decoding_acc(mset_cca_df)
    
    # store these values for easy plotting
    gcca_ntr_acc.append({**_gcca_acc['mean'], **{"n_train": n_train}})
    mset_ntr_acc.append({**_mset_acc['mean'], **{"n_train": n_train}})
    
acc_ntr_gcca = pd.DataFrame(gcca_ntr_acc).set_index("n_train")
acc_ntr_mset = pd.DataFrame(mset_ntr_acc).set_index("n_train")

#### 2. Acc vs number of samples

In [395]:
ns_range = [64, 128, 192, 256]
gcca_acc = []
mset_acc = []

for ns in ns_range:
    data_tensor_tmp = data_tensor[:, :, :ns, :]
    gcca_df, mset_cca_df = compute_gcca_msetcca_results(gcca, mset_cca, data_tensor_tmp, stim_freqs, template_idxs, ce_loss=False)
    _gcca_acc = decoding_acc(gcca_df)
    _mset_acc = decoding_acc(mset_cca_df)
    
    # store these values for easy plotting
    gcca_acc.append({**_gcca_acc['mean'], **{"Ns": ns}})
    mset_acc.append({**_mset_acc['mean'], **{"Ns": ns}})
    
acc_ns_gcca = pd.DataFrame(gcca_acc).set_index("Ns")
acc_ns_mset = pd.DataFrame(mset_acc).set_index("Ns")

In [205]:
def get_x_offsets(n, width=0.2):
    "get x offsets of bar centres for grouped bar charts in matplotlib"
    if n%2 == 0: # even
        sides = [width*x/2 for x in range(1, n+1, 2)]
        return [-1*x for x in sides[::-1]] + sides
    else: # odd
        sides = [width*x/2 for x in range(3, n+1, 2)]
        return [-1*x for x in sides[::-1]] + [0] + sides

    
def grouped_bar(x, Y):
        
    fig, ax = plt.subplots(1, figsize=(16, 6))
    for y in Y:
        plt.bar()
    
# plot bars
plt.bar(x - 0.3, df_grouped['NA_Sales'], width = 0.2, color = '#1D2F6F')
plt.bar(x - 0.1, df_grouped['EU_Sales'], width = 0.2, color = '#8390FA')
plt.bar(x + 0.1, df_grouped['JP_Sales'], width = 0.2, color = '#6EAF46')
plt.bar(x + 0.3, df_grouped['Other_Sales'], width = 0.2, color = '#FAC748')
# remove spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# x y details
plt.ylabel('Millions of copies')
plt.xticks(x, df_grouped.index)
plt.xlim(-0.5, 31)
# grid lines
ax.set_axisbelow(True)
ax.yaxis.grid(color='gray', linestyle='dashed', alpha=0.2)
# title and legend
plt.title('Video Game Sales By Platform and Region', loc ='left')
plt.legend(['NA', 'EU', 'JP', 'Others'], loc='upper left', ncol = 4)
plt.show()

[-0.5, -0.30000000000000004, -0.1, 0.1, 0.30000000000000004, 0.5]